In [ ]:
import cv2

TRACKER_TYPE = "CSRT"

multi_tracker = cv2.legacy.MultiTracker_create()
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if not ret:
    print("Failed to load video")
    cap.release()
    exit()
bboxes = []
while True:
    bbox = cv2.selectROI("Select Object", frame, fromCenter=False, showCrosshair=True)
    if bbox[2] == 0 or bbox[3] == 0:
        break
    bboxes.append(bbox)
for bbox in bboxes:
    if TRACKER_TYPE == "CSRT":
        tracker = cv2.legacy.TrackerCSRT_create()
    elif TRACKER_TYPE == "KCF":
        tracker = cv2.legacy.TrackerKCF_create()
    elif TRACKER_TYPE == "MOSSE":
        tracker = cv2.legacy.TrackerMOSSE_create()
    else:
        raise ValueError("Invalid tracker type selected")

    multi_tracker.add(tracker, frame, bbox)

cv2.destroyWindow("Select Object")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    success, boxes = multi_tracker.update(frame)

    for box in boxes:
        x, y, w, h = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Multi Object Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
